# This notebook might not run correctly. Check the python script for a complete version.

In [1]:
import scrapy
from scrapy.crawler import CrawlerProcess
from scrapy.exceptions import CloseSpider
import logging
# ignore warnings (Need to find a fix for this)
# import warnings
# warnings.filterwarnings('ignore')

In [2]:
# Disable log messages
# logging.getLogger('scrapy').propagate = False
# logging.getLogger('scrapy').setLevel(logging.WARNING)

In [2]:
class AllSpider(scrapy.Spider):
    name = 'all'
    allowed_domains = ['myxalandri.gr']
    start_urls = ['https://myxalandri.gr']

    def __init__(self):
        self.links = []
        self.max_crawled_websites = 200  # Set the maximum number of crawled websites
        self.crawled_websites = 0
        self.file = open('crawled_urls.txt', 'w')

    def parse(self, response):
        if self.crawled_websites >= self.max_crawled_websites:
            self.file.close()
            raise CloseSpider('Maximum number of crawled websites reached')

        self.links.append(response.url)
        self.crawled_websites += 1

        url = response.url + '\n'
        self.file.write(url)

        # Print the number of crawled websites
        for href in response.css('a::attr(href)'):
            yield response.follow(href, self.parse)

# Create a CrawlerProcess and start the crawling
process = CrawlerProcess()
process.crawl(AllSpider)
process.start()

2023-07-19 14:46:32 [scrapy.utils.log] INFO: Scrapy 2.9.0 started (bot: scrapybot)
2023-07-19 14:46:32 [scrapy.utils.log] INFO: Versions: lxml 4.9.3.0, libxml2 2.10.3, cssselect 1.2.0, parsel 1.8.1, w3lib 2.1.1, Twisted 22.10.0, Python 3.11.4 | packaged by conda-forge | (main, Jun 10 2023, 18:10:28) [Clang 15.0.7 ], pyOpenSSL 23.2.0 (OpenSSL 3.1.1 30 May 2023), cryptography 41.0.2, Platform macOS-13.1-x86_64-i386-64bit
2023-07-19 14:46:32 [scrapy.crawler] INFO: Overridden settings:
{}
2023-07-19 14:46:32 [scrapy.utils.log] DEBUG: Using reactor: twisted.internet.selectreactor.SelectReactor
2023-07-19 14:46:32 [scrapy.extensions.telnet] INFO: Telnet Password: a00a897f83044d1c
2023-07-19 14:46:32 [scrapy.middleware] INFO: Enabled extensions:
['scrapy.extensions.corestats.CoreStats',
 'scrapy.extensions.telnet.TelnetConsole',
 'scrapy.extensions.memusage.MemoryUsage',
 'scrapy.extensions.logstats.LogStats']
2023-07-19 14:46:32 [scrapy.middleware] INFO: Enabled downloader middlewares:
['scr

In [ ]:
# Read from the file crawled_urls.txt and remove duplicates (then print the number of files before and after removing duplicates)
with open('crawled_urls.txt', 'r') as file:
    crawled_urls = file.readlines()
    print('\n\nTotal number of crawled websites: ', len(crawled_urls))

# print the duplicated urls
print('\n\nDuplicated urls: ')
for url in crawled_urls:
    if crawled_urls.count(url) > 1:
        print(url)

# remove the duplicates from the crawled_urls list
crawled_urls = list(dict.fromkeys(crawled_urls))

# save the crawled_urls list to the file crawled_urls.txt
with open('crawled_urls.txt', 'w') as file:
    file.writelines(crawled_urls)



Total number of crawled websites:  500


Duplicated urls: 
https://myxalandri.gr

https://myxalandri.gr



In [20]:
import os
import re
from collections import defaultdict
import numpy as np

def get_sentences_from_text(text):
    # Split text into sentences using a regular expression.
    sentences = re.split(r'[.!?]', text)
    # Remove any leading/trailing whitespaces from sentences.
    sentences = [sentence.strip() for sentence in sentences if sentence.strip()]
    return sentences

def get_sentences_and_files():
    sentences_dict = defaultdict(list)
    for filename in os.listdir('myxalandri/txt_files/'):
        # print('--', filename)
        if filename.endswith('.txt'):
            with open('myxalandri/txt_files/'+filename, 'r', encoding='utf-8') as file:
                content = file.read()
                # Extract the description section using regex.
                description_match = re.search(r'Description:(.+)', content, re.DOTALL)
                if description_match:
                    description_text = description_match.group(1)
                    sentences = get_sentences_from_text(description_text)
                    for sentence in sentences:
                        sentences_dict[sentence].append(filename)
    return sentences_dict

def print_duplicates(sentences_dict):
    for sentence, files in sentences_dict.items():
        # if length of sentence is greater than 10 characters:
        if len(sentence) > 50:
            if len(np.unique(files)) > 1 and len(files) > 1:
                print(f"Sentence: {sentence}")
                print("Found in files:", ", ".join(np.unique(files)))
                print()

if __name__ == "__main__":
    sentences_dict = get_sentences_and_files()
    print_duplicates(sentences_dict)


Sentence: Αντί εισιτηρίου στις παραστάσεις με ελεύθερη είσοδο, προαιρετική συνεισφορά σε τρόφιμα, φάρμακα και άλλα είδη πρώτης ανάγκης για τους πρόσφυγες και τις κοινωνικές δομές της πόλης
Found in files: 1.txt, 22.txt, 24.txt, 28.txt, 29.txt, 31.txt

Sentence: ), προσφέρει στους δημότες του πληθώρα ευκαιριών καλλιτεχνικής έκφρασης και απόλαυσης, απασχολώντας μεγάλο αριθμό καλλιτεχνών με διάφορες μορφές απασχόλησης
Found in files: 126.txt, 128.txt

Sentence: Με το παραπάνω σκεπτικό, κάθε ενέργεια που μειώνει ηθικά και αξιακά την καλλιτεχνική εκπαίδευση και θίγει τα δικαιώματα των λειτουργών της, όπως το πρόσφατο Π
Found in files: 126.txt, 128.txt

Sentence: Η Γκαλερί Χρυσόθεμις παρουσιάζει ομαδική έκθεση ζωγραφικής
Found in files: 106.txt, 120.txt, 194.txt

Sentence: comfacebook: Γκαλερί Χρυσόθεμις Χαλάνδριinstagram: gallery_chryssothemis Ημέρες και ώρες λειτουργίας: Τρίτη: 11:00-14:00 &amp;amp; 17:30-20:00 Πέμπτη: 11:00-14:00 &amp;amp; 17:30-20:00 Παρασκευή: 11:00-14:00 &amp;amp; 17:3

In [12]:
! pip install numpy

  Obtaining dependency information for numpy from https://files.pythonhosted.org/packages/e3/7e/0b072c21f4feefb2d89600956af307db29fb7df695cbe6e145de91643155/numpy-1.25.1-cp311-cp311-macosx_10_9_x86_64.whl.metadata
Using cached numpy-1.25.1-cp311-cp311-macosx_10_9_x86_64.whl (20.0 MB)
